In [1]:
import pandas as pd
import os

# Timestep Log File to Excel

The timestep log is very useful When debugging an InfoWorks ICM model, however, the log file usually will have thousands of lines and extracting the useful information can be a challenge.

Using the code block below, the count tables showing the nodes and links with trouble in calculation are exported into an Excel spreadsheet. Each tab is a table ordered with the object with the highest number of count of iterations.

For example, the tabs are named as "line no-Table name"

- 93-Unconverged link depth
- 98-Unconverged nodes coun
- 1084-Link depth fail coun



In [57]:
def token(l):
    if ' counts:' in l:
        return ('counts header', l.strip())
    else:
        return ('line', l)
    

def get_table(log_path):
    ct = 0
    with open(log_path) as f:
        
        tables = {}
        table_name = None
        for l in f:
            ct += 1
            a, b = token(l)
            if a == 'counts header':
                b = '{}-{}'.format(ct, b.replace(':', ''))[:25]
                tables[b] = []
                if table_name is None:
                    table_name = b
            elif a == 'line':
                if l =='\n':
                    table_name = None
                else:
                    if table_name:
                        tables[table_name].append(l.strip().split(' '))
                # print(l)
        return tables

def process_tables(tables):
    results = {}
    for fld in tables:
        results[fld] = pd.DataFrame(tables[fld], columns=['ID', 'count'])
        results[fld]['count'] = pd.to_numeric(results[fld]['count'])
    return results

def save_tables(tables, excel_path):
    with pd.ExcelWriter(excel_path) as writer:
        for fld in tables:
            tables[fld].sort_values(by=['count'], ascending=False).to_excel(writer, sheet_name=fld, index=False)


def log_to_excel(log_path, excel_path):
    tables = process_tables(get_table(log_path))
    save_tables(tables, excel_path)

In [59]:
# step 1 turn on timestep log in the RUN
# step 2 run the simulation
# step 3 export the log to a file
# step 4 set up the log path, and the excel_path and run this cell
log_path = './data/sim.log'
excel_path = './data/log.xlsx'
log_to_excel(log_path, excel_path)

In [61]:
# Here is an example to read the tables into a variable
tables = process_tables(get_table(log_path))
for fld in tables:
    print(fld)
    print(tables[fld].head())

93-Unconverged link depth
         ID  count
0   INLET.1      8
1   INLET.2      1
2  OUTLET.1     12
98-Unconverged nodes coun
       ID  count
0  OUTLET     19
1084-Link depth fail coun
         ID  count
0  FM_END.1     16
1   INLET.1     71
2   INLET.2      5
3  OUTLET.1      4
1090-Link flow fail count
         ID  count
0  FM_END.1     11
1   INLET.1     27
2  OUTLET.1     51
1095-Node fail counts
       ID  count
0  FM_END     44
1   INLET     11
2  OUTLET     61
1100-Unconverged link dep
         ID  count
0  FM_END.1    317
1   INLET.1   1098
2   INLET.2    112
3  OUTLET.1    119
1106-Unconverged link flo
         ID  count
0  FM_END.1    433
1   INLET.1    419
2   INLET.2      2
3  OUTLET.1    262
1112-Unconverged nodes co
       ID  count
0  FM_END    541
1   INLET    446
2  OUTLET    844
